In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [4]:
import csv
import re
data = []

with open('SentiNews_paragraph-level.txt', 'r') as f:
  lineReader = csv.reader(f, delimiter=',', quotechar="\"")
  for row in lineReader:
    if row:
      row = ' '.join(row)
      elementi = row.split('\t')
      sentence = elementi[2]
      sentiment = elementi[11]
      sentiment = re.sub(r'[^\w ][0-9]+', "", sentiment)
      data.append({'text': sentence, 'sent': sentiment})

In [5]:
data = data[1:]

In [6]:
import pandas as pd
df = pd.DataFrame(data, index=None, columns=['text', 'sent'])

In [7]:
df['sent'].value_counts()

neutral     40358
negative    18268
positive    10781
Name: sent, dtype: int64

In [8]:
df.sample(15)

,text,sent
7314,Bivši predsednik uprave Ivan Majcen Foto Kanal A,neutral
25692,Vlaganje v okolju prijazne projekte je kratko...,neutral
67603,“Prisilno odločanje o zadevah pred letom 2007 ...,neutral
49239,KRKA +0 00 % 57 50,neutral
49246,Klepet o Pravi ideji! z Edito Cetinski Malnar,neutral
27873,Foto: Dreamstime,neutral
59275,Vroče tudi na Hrušici in v Tupaličah,negative
34432,Zadnje četrtletje se je na Ljubljanski borzi z...,neutral
41857,Vozila kljub težavam varna?,neutral
968,Spomnimo ...,neutral


In [9]:
df['text'] = df['text'].apply(lambda x: ' '.join(x.lower().split()))

In [10]:
df['text'] = df['text'].apply(lambda x: ' '.join(x.split()))

In [11]:
import re
df['text'] = df['text'].apply(lambda x: re.sub(r'[^\w ]+', "", x))

In [12]:
df.shape

(69407, 2)

In [13]:
df = df.dropna()

In [14]:
df.shape

(69407, 2)

In [15]:
prazniS = []
for i in range(len(df['sent'])):
  s = df['sent'][i]
  if not s:
    prazniS.append(i)

In [16]:
len(prazniS)

0

In [17]:
prazniText = []
for i in range(len(df['text'])):
  s = df['text'][i]
  if not s:
    prazniS.append(i)

In [18]:
len(prazniText)

0

In [19]:
train = df[:25000]

In [20]:
test = df[25001:35715]

In [21]:
train.shape

(25000, 2)

In [22]:
test.shape

(10714, 2)

In [23]:
#from google.colab import files
train.to_csv('sentTrain.csv', encoding = 'utf-8-sig', index=False) 
test.to_csv('sentTest.csv', encoding = 'utf-8-sig', index=False)

In [24]:
!pip install transformers datasets sklearn numpy torch torchvision

In [25]:
from datasets import load_dataset, load_metric

In [26]:
with open("sentTrain.csv",'r') as f:
    with open("sentTrainHeadless.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
          f1.write(line)

In [27]:
with open("sentTest.csv",'r') as f:
    with open("sentTestHeadless.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
          f1.write(line)

In [28]:
dataset = load_dataset(
    'csv',
    data_files={
        'train': 'sentTrainHeadless.csv',
        'test': 'sentTestHeadless.csv'
    },
    column_names = ['sentence', 'label']
)

Using custom data configuration default-adec848ed285d3eb


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/ncirar/.cache/huggingface/datasets/csv/default-adec848ed285d3eb/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
metric = load_metric('glue', 'sst2')

In [30]:
from transformers import AutoTokenizer

In [31]:
tokenizer = AutoTokenizer.from_pretrained(
    'EMBEDDIA/crosloengual-bert',
    use_fast=True
)

In [32]:
tokenizer(['hello', 'world'])

{'input_ids': [[103, 17592, 1169, 104], [103, 2329, 104]], 'token_type_ids': [[0, 0, 0, 0], [0, 0, 0]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1]]}

In [33]:
label2id = {'positive': 2, 'neutral': 1, 'negative': 0}
id2label = ['negative', 'neutral', 'positive']

In [34]:
def preprocess(examples):
  result = tokenizer(examples['sentence'], truncation=True, max_length=512)
  result['label'] = [label2id[l] for l in examples['label']]
  return result

In [35]:
encoded_dataset = dataset.map(preprocess, batched=True, load_from_cache_file=False)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [36]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np

In [37]:
model = AutoModelForSequenceClassification.from_pretrained(
    'EMBEDDIA/crosloengual-bert',
    num_labels=3
)

Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model chec

In [38]:
args = TrainingArguments(
    "tweet-sentiment",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    )

In [39]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [40]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=None,
    )

In [41]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/ncirar/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4689


Epoch,Training Loss,Validation Loss,Accuracy
1,0.585700,0.931086,0.630577
2,0.460900,0.947868,0.650364
3,0.337700,1.061188,0.649897


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10714
  Batch size = 16
Saving model checkpoint to tweet-sentiment/checkpoint-1563
Configuration saved in tweet-sentiment/checkpoint-1563/config.json
Model weights saved in tweet-sentiment/checkpoint-1563/pytorch_model.bin
tokenizer config file saved in tweet-sentiment/checkpoint-1563/tokenizer_config.json
Special tokens file saved in tweet-sentiment/checkpoint-1563/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation **

TrainOutput(global_step=4689, training_loss=0.4796912782505952, metrics={'train_runtime': 1053.7657, 'train_samples_per_second': 71.173, 'train_steps_per_second': 4.45, 'total_flos': 4900285958973552.0, 'train_loss': 0.4796912782505952, 'epoch': 3.0})

In [42]:
eval_results = trainer.evaluate()
print(eval_results)

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10714
  Batch size = 16


{'eval_loss': 0.9478681087493896, 'eval_accuracy': 0.6503640097069255, 'eval_runtime': 32.1602, 'eval_samples_per_second': 333.145, 'eval_steps_per_second': 20.833, 'epoch': 3.0}


In [43]:
trainer.save_model(output_dir='tweet-sentiment-model')

Saving model checkpoint to tweet-sentiment-model
Configuration saved in tweet-sentiment-model/config.json
Model weights saved in tweet-sentiment-model/pytorch_model.bin
tokenizer config file saved in tweet-sentiment-model/tokenizer_config.json
Special tokens file saved in tweet-sentiment-model/special_tokens_map.json


In [44]:
model = AutoModelForSequenceClassification.from_pretrained('tweet-sentiment-model')

loading configuration file tweet-sentiment-model/config.json
Model config BertConfig {
  "_name_or_path": "tweet-sentiment-model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 49601
}

loading weights file tweet-sentiment-model/pytorch_model.bin
All

In [45]:
examples=['Rezultati za prejšnje leto so res pohvale vredni.',
          'Najlepša hvala za pomoč, zelo sem hvaležen.',
          'Neumni politiki nimajo pojma.', 
          'Če me ne pustiš pri miru, te bom udaril!',
          'Jutri bo deževalo.',
          'Ne maram mleka.']

inputs = tokenizer(examples, padding='longest', return_tensors="pt")
outputs = model(**inputs)
probs = outputs[0].detach().numpy()
for i in range(len(examples)):
    print(examples[i],'\t', id2label[np.argmax(probs[i])])

Rezultati za prejšnje leto so res pohvale vredni. 	 positive
Najlepša hvala za pomoč, zelo sem hvaležen. 	 positive
Neumni politiki nimajo pojma. 	 negative
Če me ne pustiš pri miru, te bom udaril! 	 negative
Jutri bo deževalo. 	 negative
Ne maram mleka. 	 negative


In [63]:
corpus = []
transkript = []
dvajset = []
devetnajst = []
osemnajst = []
sedemnajst = []
sestnajst = []

i=0
with open('dataframe.csv', 'r') as f:
  lineReader = csv.reader(f, delimiter=',', quotechar="\"")
  next(f)
  for row in lineReader:
    transkript.append(row[0])
    t = row[1].split('-')
    leto = t[0]
    mesec = t[1]
    corpus.append({'text': row[0], 'leto': leto, 'mesec': mesec})
    if leto == '2020':
        dvajset.append({'text': row[0], 'leto': leto, 'mesec': mesec})
    elif leto == '2019':
        devetnajst.append({'text': row[0], 'leto': leto, 'mesec': mesec})
    elif leto == '2018':
        osemnajst.append({'text': row[0], 'leto': leto, 'mesec': mesec})
    elif leto == '2017':
        sedemnajst.append({'text': row[0], 'leto': leto, 'mesec': mesec})
    else:
        sestnajst.append({'text': row[0], 'leto': leto, 'mesec': mesec})

In [64]:
df = pd.DataFrame(corpus, index=None, columns=['text', 'leto', 'mesec'])

In [65]:
df.sample(5)

,text,leto,mesec
25058,mag branko grims lepo prosim izogibate pripomb,2016,05
6409,prosim zaključite,2015,03
56234,hvala lepa gospod šušmelj odpovedal razpravi i...,2018,12
260,dam besedo naprej ponovno odprem razpravo očit...,2014,08
3140,spoštovani predsednik cenjeni poslanci kratko ...,2014,12


In [66]:
df['leto'].value_counts()

2015    15671
2016    14724
2017    13571
2019    11877
2018     8515
2020     6601
2014     4163
Name: leto, dtype: int64

In [67]:
dva = pd.DataFrame(dvajset, index=None, columns=['text', 'leto', 'mesec'])
devet = pd.DataFrame(devetnajst, index=None, columns=['text', 'leto', 'mesec'])
osem = pd.DataFrame(osemnajst, index=None, columns=['text', 'leto', 'mesec'])
sedem = pd.DataFrame(sedemnajst, index=None, columns=['text', 'leto', 'mesec'])
sest = pd.DataFrame(sestnajst, index=None, columns=['text', 'leto', 'mesec'])

In [72]:
textSest = list(sest['text'])
type(textSest)

list

In [76]:
examples=textSest

inputs = tokenizer(examples, padding='longest', return_tensors="pt")
outputs = model(**inputs)
probs = outputs[0].detach().numpy()
for i in range(len(examples)):
    print(examples[i],'\t', id2label[np.argmax(probs[i])])

Token indices sequence length is longer than the specified maximum sequence length for this model (5386 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: [enforce fail at alloc_cpu.cpp:73] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 571789479936 bytes. Error code 12 (Cannot allocate memory)